# Defining AKS Resources with Terraform

In this lesson, we'll look in more detail at Azure Kubernetes Service (AKS) and Terraform, focusing on how to combine these powerful technologies to efficiently manage and orchestrate Kubernetes clusters on Azure.

## Setting up the Development Environment

Before we start defining Azure Kubernetes Service (AKS) resources with Terraform, we need to ensure that your development environment is properly configured.

To create a robust and fully functional AKS cluster, we need to consider various components that make up the infrastructure. These components include:

- **Virtual Network (VNet)**: The networking foundation for your AKS cluster, which includes subnets for worker nodes, control plane nodes, and any additional network resources
- **Subnets**: Specific network segments within the VNet, including those for worker nodes and control plane nodes
- **Network Security Groups (NSGs)**: Security rules to control inbound and outbound traffic to resources in the subnets

These components are essential for configuring the network infrastructure to support the AKS cluster. Thus, this Terraform project will not only need to contain a module for defining the AKS cluster resource itself, but an additional networking module that will focus on creating the VNet, subnets, and NSGs required for your AKS environment.

Start by creating a dedicated directory for your Terraform project. This directory will contain all your Terraform configuration files.

```shell
mkdir aks-terraform
cd aks-terraform
```

Inside this directory, create two subdirectories: `networking-module` and `aks-cluster-module` to represent the networking services and the AKS cluster configuration, respectively.

```shell
mkdir networking-module
mkdir aks-cluster-module
```

By creating these directories, we are preparing to organize our Terraform modules in a structured manner. Now let's move on to the next sections, where we will define the necessary modules in detail and incorporate them into the main Terraform configuration.

## Networking Services Module Configuration

> For the networking setup, please refer to the configuration details from the previous lesson, as we will reuse the module setup in the `networking-module` directory of this project. However, there are some key differences in this setup compared to the previous example:

- For the AKS setup, you need to create two separate subnets, rather than a single one (by creating two different `azurerm_subnet` blocks):
    - Control Plane Subnet: Dedicated to the Kubernetes control plane nodes
    - Worker Nodes Subnet: Dedicated to the Kubernetes worker nodes
- The AKS setup requires specific NSG rules tailored to the needs of the Kubernetes cluster. The earlier setup included NSG rules for HTTP, HTTPS and a range of node ports. For AKS setup you will need to define the following:
    - `kube-apiserver` Port (TCP/443): Allows secure communication with the Kubernetes API server, which is essential for managing the cluster
    - SSH Port (TCP/22) (Optional): This rule allows SSH access to the nodes if you need to perform administrative tasks or troubleshooting. It is optional and should only be added if necessary for your use case.

In summary, the key differences are the addition of multiple subnets for better segregation of control plane and worker nodes, and the introduction of specific NSG rules that cater to the operational requirements of the AKS cluster, as opposed to more general network access rules.

The NSG rules should look like this:

```hcl
# Allow inbound traffic to kube-apiserver (TCP/443) from your public IP address
resource "azurerm_network_security_rule" "kube_apiserver" {
  name                        = ""<your_nsg_rule1_name>"
  priority                    = 1001
  direction                   = "Inbound"
  access                      = "Allow"
  protocol                    = "Tcp"
  source_port_range           = "*"
  destination_port_range      = "443"
  source_address_prefix       = "*"  
  destination_address_prefix  = "*"
  resource_group_name         = azurerm_resource_group.networking.name
  network_security_group_name = azurerm_network_security_group.aks_nsg.name
}

# Allow inbound traffic for SSH (TCP/22) - Optional
resource "azurerm_network_security_rule" "ssh" {
  name                        = "<your_nsg_rule2_name>"
  priority                    = 1002
  direction                   = "Inbound"
  access                      = "Allow"
  protocol                    = "Tcp"
  source_port_range           = "*"
  destination_port_range      = "22"
  source_address_prefix       = "*"  
  destination_address_prefix  = "*"
  resource_group_name         = azurerm_resource_group.networking.name
  network_security_group_name = azurerm_network_security_group.aks_nsg.name
}
```

> Additionally, we will define a `outputs.tf` file for our new networking module, allowing us to make available the ouputs of this module to the AKS cluster module.

Here's an example of how you can define output variables:

```hcl
# networking-module/outputs.tf

output "vnet_id" {
  description = "ID of the Virtual Network (VNet)."
  value       = <vnet-resource-type>.<vnet-resource-identifier>.id
}

output "control_plane_subnet_id" {
  description = "ID of the control plane subnet."
  value       = <subnet-resource-type>.<control-plane-subnet-identifier>.id
}

output "worker_node_subnet_id" {
  description = "ID of the worker node subnet."
  value       = <subnet-resource-type>.<worker-node-subnet-identifier>.id
}

# Define more output variables as needed...
```

> Within the `value` argument of each `output` block you will have to replace the `<resource-type>` and `<resource-identifier>` with the correct resource type and identifier for each resource. You can find these values in the previously created `main.tf` where you provisioned your resources.

Let's look at an example, for the VNet resource:

```hcl
# Define the Virtual Network (VNet) for the AKS cluster
resource "azurerm_virtual_network" "aks_vnet" {
  name                = "<your_vnet_name>"
  address_space       = var.vnet_address_space
  location            = azurerm_resource_group.networking.location
  resource_group_name = azurerm_resource_group.networking.name
}
```

The `output` block should look like this:

```hcl
output "vnet_id" {
  description = "ID of the Virtual Network (VNet)."
  value       = azurerm_virtual_network.aks_vnet.id
}
```
In this case `<resource-type>` corresponds to `azurerm_virtual_network` and `<resource-identifier>` corresponds to `aks_vnet` as we can see from the VNet `resource` block.

The `vnet_id` output variable provides the ID of the VNet created by the networking module. This output variable can then be used in the AKS cluster module to specify the VNet in which the AKS cluster should be created. 

The `control_plane_subnet_id` output variables provides the ID of the control plane subnet within the VNet. The control plane subnet is where the control plane components of the AKS cluster, such as the `kube-apiserver`, are deployed. You can use this output variable to configure the AKS cluster to use the specified subnet for its control plane. 

The `worker_node_subnet_id` output variables provides the ID of the worker node subnet within the VNet. The worker node subnet is where the AKS cluster's worker nodes are deployed. You can use this output variable to configure the AKS cluster to use the specified subnets for its worker nodes.


### Review and Apply Configuration

Run `terraform init` to set up the Terraform workspace and download the necessary provider plugins and modules. 

We will proceed with defining the AKS cluster module's configuration in the next section, and then we will apply both modules together, from the main project directory.

## Define AKS Cluster Module Resources

In this section, we will define the Terraform configuration for creating an AKS cluster. Before we start defining the AKS cluster configuration, navigate to the module directory for your AKS resources `aks-cluster-module`. 

Inside the AKS module directory, create the following files:

- `variables.tf`: This file will define the input variables that allow customization of the AKS cluster
- `terraform.tfvars`: In this file, you will manage sensitive information such as credentials
- `outputs.tf`: In this file, you can specify any output values that you want to retrieve after provisioning the AKS cluster
- `main.tf`: The main configuration file where we will define the actual AKS cluster resources

### Define Input Variables (`variables.tf`) 

To create a flexible and customizable AKS cluster configuration, we will define input variables that can be customized when using this module. Here are the input variables for this module:

```hcl
# aks-cluster/variables.tf

variable "aks_cluster_name" {
  ...
}

variable "location" {
  ...
}

variable "service_principal_client_id" {
  ...
}

variable "service_principal_client_secret" {
  ...
}

# Input variables from the networking module
variable "resource_group_name" {
  ...
}

variable "vnet_id" {
  ...
}

...
```
> Inside the `variables.tf` file of this module, you should make sure to include all the output variables from the networking module. As an example `resource_group_name` and `vnet_id` have already been included. Make sure to add the rest of them including `control_plane_subnet_id`, `worker_node_subnet_id` and `aks_nsg_id`.

Here's an explanation of the new input variables defined in the AKS cluster module configuration. Later, you will use the output values from networking module as input variables in the AKS cluster module. This allows you to pass data and share information between modules in a structured and controlled manner.

- The `aks_cluster_name` variable specifies the name of the AKS cluster that will be created
- The `location` defines the Azure region where the AKS cluster will be created
- The `service_principal_client_id` is the Client ID of the service principal used for authenticating and managing the AKS cluster
- The `service_principal_client_secret` specifies the Client Secret associated with the service principal used for AKS cluster authentication

We will use these input variables to configure and create the AKS cluster resources in the `main.tf` file.

> IMPORTANT: When working with the AiCore-generated Azure credentials for the specialisation project, you will use a *Managed Identity* instead of a Service Principal to create the AKS cluster. Managed Identity is a feature that provides an identity for Azure services to use when connecting to resources that support Entra ID authentication. Unlike a Service Principal, which required explicit management of credentials, a Managed Identity is automatically managed by Azure and eliminates the need for storing secrets.

This means for the project, you will need to replace the `service_principal_client_id` and `service_principal_client_secret` variables with a `managed_identity_id` variable. Here's how you would adjust the `variables.tf`:

```hcl
variable "managed_identity_id" {
  description = "The Managed Identity ID used for the AKS cluster."
  type        = string
}
```

### Define Configuration Values (`terraform.tfvars`)

To provide the sensitive values for the input variables defined in `variables.tf`, you will use a `terraform.tfvars` file. This file allows you to specify concrete values for variables that Terraform will use during planning and application of your configuration.

In this example, where you are using a Service Principal for authentication, the `terraform.tfvars` might look like this:

```hcl
service_principal_client_id = "your-service-principal-client-id"
service_principal_client_secret = "your-service-principal-client-secret"
```

> For the specialisation project, when you will be using a Managed Identity instead, the `terraform.tfvars` will look slight different:

```hcl
managed_identity_id = "your-managed-identity-id"
```

### Define AKS Cluster Module

In the `main.tf`, you can now define the configuration for the AKS cluster module.

```hcl
# aks-cluster/main.tf
# Create the AKS cluster
resource "azurerm_kubernetes_cluster" "aks_cluster" {
  name                = var.aks_cluster_name
  location            = var.cluster_location
  resource_group_name = var.resource_group_name
  dns_prefix          = "example"

  default_node_pool {
    name       = "default"
    node_count = 1
    vm_size    = "Standard_DS2_v2"
    enable_auto_scaling = true
    min_count = 1
    max_count = 2
  }

  service_principal {
    client_id     = var.service_principal_client_id
    client_secret = var.service_principal_client_secret
  }
}

```
Let's break down the configuration block for creating the AKS cluster:

```# Create the AKS cluster
resource "azurerm_kubernetes_cluster" "aks_cluster" {
  name                = var.aks_cluster_name
  location            = var.cluster_location
  resource_group_name = var.resource_group_name
```

This block specifies the information we already explained in detail when defining the `variables.tf` for this module.

```hcl
  default_node_pool {
    name       = "default"
    node_count = 1
    vm_size    = "Standard_DS2_v2"
    enable_auto_scaling = true
    min_count = 1
    max_count = 2
  }
```

The `default_node_pool` block defines the default node pool for the cluster:

- `name`: Specifies the name of the default node pool, which is `default` in this case
- `node_count`: Sets the initial number of nodes in the node pool to 1
- `vm_size`: Specifies the virtual machine size for the nodes in the pool, which is `Standard_DS2_v2` in this example
- `enable_auto_scaling`: Enables auto-scaling for the node pool
- `min_count`: Sets the minimum number of nodes to 1
- `max_count`: Sets the maximum number of nodes to 2

```hcl
  service_principal {
    client_id     = var.service_principal_client_id
    client_secret = var.service_principal_client_secret
  }
}
```

Finally, the `service_principal` blocks provides the authentication details for the AKS cluster. These are the same credentials you use when defining an Azure provider in your configuration. You should have access to this if you have provisioned a Service Principal in the previous lessons.

> For the specialisation project you will need to replace the `service_principal` block with an `identity` block in your `main.tf` file:

```hcl
  identity {
      type = "UserAssigned"
      identity_ids = [
        var.managed_identity_id
      ]
    }
```

The `identity` block is used to configure Managed Identity for the AKS cluster. 

### Define Output Variables (`outputs.tf`) 

In the `outputs.tf` define the output variables for the AKS cluster module:

```hcl
output "aks_cluster_name" {
  description = "Name of the AKS cluster."
  value       = <kubernetes-cluster-resource-type>.<kubernetes-cluster-resource-identifier>.name
}

output "aks_cluster_id" {
  description = "ID of the AKS cluster."
  value       = <kubernetes-cluster-resource-type>.<kubernetes-cluster-resource-identifier>.id
}

output "aks_kubeconfig" {
  description = "Kubeconfig file for accessing the AKS cluster."
  value       = <kubernetes-cluster-resource-type>.<kubernetes-cluster-resource-identifier>.kube_config_raw
}
```
> Within the `value` argument of each `output` block you will have to replace the `<resource-type>` and `<resource-identifier>` with the correct resource type and identifier for each resource. You can find these values in the previously created `main.tf` where you provisioned your resources.

These output variables provide information about the AKS cluster, including its name, ID, and the `kubeconfig` file for accessing the cluster using `kubectl`.

### Review and Apply Configuration

Run `terraform init` to set up the Terraform workspace and download the necessary provider plugins and modules. 

In the next section we will apply both modules together, from the main project directory.

## Define the Main Configuration

In this section, we will define the main configuration for your project, which will utilize the networking module and the AKS cluster module to provision the necessary Azure resources.

Begin by navigating to the project directory `aks-terraform`. Here we will begin defining your project's main configuration file, `main.tf`, by defining the Azure provider block for authentication. This block should be included at the beginning of the configuration file. 

```hcl
terraform {
  required_providers {
    azurerm = {
      source  = "hashicorp/azurerm"
      version = "=3.95.0"
    }
  }
}

provider "azurerm" {
  features {}
  skip_provider_registration = true
  client_id       = var.client_id
  client_secret   = var.client_secret
  subscription_id = "your-subscription-id"
  tenant_id       = "your-tenant-id"
}
```
This provider block specifies that you will be using Azure as your cloud provider. It uses variables for the `client_id` and `client_secret`, which we will store in environment variables to avoid hardcoding them. First we need to add them to the main configuration's `variables.tf` as follows:

```hcl
#variables.tf

variable "client_id" {
  description = "Access key for the provider"
  type        = string
  sensitive   = true
}

variable "client_secret" {
  description = "Secret key for the provider"
  type        = string
  sensitive   = true
}

variable "subscription_id" {
  description = "Azure subscription ID"
  type        = string
}

variable "tenant_id" {
  description = "Azure tenant ID"
  type        = string
}

```

Create a `terraform.tfvars` file in the main project directory to securely store the values for these variables:

```hcl
# terraform.tfvars

client_id       = "<your-client-id-here>"
client_secret   = "<your-client-secret-here>"
subscription_id = "<your-subscription-id-here>"
tenant_id       = "<your-tenant-id-here>"
```

>IMPORTANT: The `terraform.tfvars` file should not be checked into version control to keep sensitive information secure.  Make sure to add it to your `.gitignore`.


### Use the Networking Module

Now, let's use the networking module you created to provision the networking resources needed for your AKS cluster. In your `main.tf`, include the following code:

```hcl
# main.tf

module "networking" {
  source = "./networking-module"

  # Input variables for the networking module
  location           = "UK South"
  vnet_address_space = ["10.0.0.0/16"]

  # Define more input variables as needed...
}
```
This code imports the networking module and provides values for the required input variables. If the required variables already have a default value encoded in `variables.tf` and you don't want to overwrite these values, you can simply call the module using the `source` filed, without the additional variable fields.

### Use the AKS Cluster Module

Next, utilize the AKS cluster module to define and provision your AKS cluster. Add the following code to your `main.tf`:

```hcl
# main.tf

module "aks_cluster" {
  source = "./aks-cluster-module"

  # Input variables for the AKS cluster module
  aks_cluster_name           = "terraform-aks-cluster"
  cluster_location           = var.location
  service_principal_client_id = var.client_id
  service_principal_client_secret = var.client_secret

  # Input variables referencing outputs from the networking module
  resource_group_name         = module.networking.resource_group_name
  vnet_id                     = module.networking.vnet_id
  control_plane_subnet_id     = module.networking.control_plane_subnet_id
  worker_node_subnet_id       = module.networking.worker_node_subnet_id
  aks_nsg_id                  = module.networking.aks_nsg_id

  # Define more input variables as needed...
}
```

> IMPORTANT: For the specialisation project, you will need to replace the `service_principal_client_id` and `service_principal_client_secret` with the `managed_identity_id`.

In this code, you import the AKS cluster module and provide values for the input variables required by the module. Note you will once again need to add input variables to `variables.tf`, and sensitive variables values to `terraform.tfvars`, for your service principal ID and secret, or managed identity ID for the specialisation project.

### Review and Apply Configuration

**IMPORTANT: DO NOT ACTUALLY RUN THE `terraform apply` COMMAND.**
**Provisioning an AKS cluster is a very expensive process. If you are using an Azure Free Tier account it will diminish your free credits very quickly. If you are completing your specialisation project, you can continue with the remaining steps and provision your AKS cluster.**

After defining the main configuration file, you can now review the changes and plan the deployment:

```shell
terraform init
terraform plan
```

If the plan looks satisfactory, you can apply the configuration to create the networking resources and the AKS cluster using `terraform apply`. 

> If you run into any permissions issues when running `terraform apply` make sure to check your Service Principal has the correct permissions assigned to it. You can do this by navigating to the **Subscriptions** web page, identifying your own subscription. From here access the **Access control (IAM)** page. Under **Role assignments**, you should see your Service Principal with the role **Contributor** assigned to it. Like in the image below:

<p align=center><img src=images/IAMRole.png width=950 height=450></p>

In the example above, the Service Principal is called `myAppMaya`.

> Again, if you are running this as part of the specialisation project the issue above should'nt apply as all the necessary permissions have already been assigned to your AiCore-generated Azure account.

With these networking configurations and the AKS cluster in place, your infrastructure will be set up according to your specifications. Be mindful that this might take a couple of minutes. Congratulations on deploying your first AKS cluster using Terraform!

## Accessing the AKS Cluster

Once you've deployed your AKS cluster, you'll need to access it to manage and deploy applications. In this section, we'll guide you on how to retrieve the `kubeconfig` and use `kubectl` to interact with the cluster.

### Retrieve `kubeconfig`

> Before you retrieve the configuration file, you need to make sure you are logged into the correct Azure account, which for the specialisation project is the account that has been generated for you and for which the credentials have been emailed to you at the beginning of the project.

The `kubeconfig` is a configuration file that allows you to connect to your AKS cluster securely. To retrieve the `kubeconfig`, follow these steps:

```shell
# Use the Azure CLI to get the AKS cluster credentials
az aks get-credentials --resource-group <your-resource-group> --name <your-aks-cluster-name>
```

Replace `<your-resource-group>` and `<your-aks-cluster-name>` with the appropriate values for your AKS cluster.

The above command will fetch the `kubeconfig` and automatically merge it into your local `~/.kube/config` file, which is the default location for `kubeconfig`. Ensure that you have Azure CLI installed and configured with the necessary permissions.

<p align=center><img src=images/kubeFile.png width=600 height=350></p>

After retrieving the `kubeconfig`, you can start using `kubectl` to interact with your AKS cluster. For example check the status of your nodes you can run `kubectl get nodes`.

With these steps, you can easily access and manage your AKS cluster using `kubectl`, making it a powerful tool for deploying and managing Kubernetes workloads.

## Clean Up Resources

When you no longer need your AKS cluster and associated resources, it's crucial to clean up to avoid unnecessary costs. Use Terraform's `terraform destroy` command to deprovision AKS and related resources while maintaining infrastructure as code (IaC) principles.

Before performing any destructive actions, double-check that you have backups of critical data and configurations.

## Conclusion

In this notebook, we explored the process of provisioning, configuring, and managing Azure Kubernetes Service (AKS) clusters using Terraform. By following Infrastructure as Code (IaC) principles, modularization, and best practices, you learned how to efficiently create and maintain AKS clusters to run containerized applications at scale.